## Lesson 2: Connecting with a CRM

In [1]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources for the lesson.
# It does this by quickly running through all the code from the previous lessons.

!sh ./ro_shared_data/reset.sh
%run ./ro_shared_data/lesson_2_prep.py lesson2

import os

agentId = os.environ['BEDROCK_AGENT_ID']
agentAliasId = os.environ['BEDROCK_AGENT_ALIAS_ID']
region_name = 'us-west-2'
lambda_function_arn = os.environ['LAMBDA_FUNCTION_ARN']

Resetting environment (if nessesary)
Found: mugs-customer-support-agent
Deleting alias: MyAgentAlias (ID: FIJU8NIAID)
Deletion initiated for alias: MyAgentAlias
Alias MyAgentAlias has been successfully deleted.
Deleting alias: AgentTestAlias (ID: TSTALIASID)
Error deleting alias AgentTestAlias: An error occurred (ValidationException) when calling the DeleteAgentAlias operation: 1 validation error detected: Value 'TSTALIASID' at 'agentAliasId' failed to satisfy constraint: Member must satisfy regular expression pattern: (?!\bTSTALIASID\b)[0-9a-zA-Z]+
Deleting agent: mugs-customer-support-agent (ID: INPYU0VLYP)
Deletion initiated for agent: mugs-customer-support-agent
Waiting for agent mugs-customer-support-agent to be deleted...
Waiting for agent mugs-customer-support-agent to be deleted...
Agent mugs-customer-support-agent has been successfully deleted.
Agent reset process completed.
Deleting Lambda function: dlai-support-agent-ZJN1F
Lambda function dlai-support-agent-ZJN1F deleted suc

## Start of lesson

In [2]:
import boto3
import uuid
from helper import *

In [3]:
sessionId = str(uuid.uuid4())
message = "My name is Mike, my mug is broken and I want a refund."

In [4]:
invoke_agent_and_print(
    agentId=agentId, 
    agentAliasId=agentAliasId, 
    inputText=message, 
    sessionId=sessionId
)

User: My name is Mike, my mug is broken and I want a refund.

Agent: I'm sorry, but I don't have the ability to directly process your
       refund request for the broken mug. As an AI assistant, I'm not
       able to access the necessary tools and information to handle
       customer service issues like this. The best I can do is to
       route your request to one of our human customer support agents
       who can properly investigate the issue and provide you with the
       appropriate resolution. Please hold while I transfer you to an
       agent who can assist you further with your refund.

Session ID: 49d5c0fb-1f19-4a19-9244-1a20e1863757


In [5]:
bedrock_agent = boto3.client(service_name = 'bedrock-agent', region_name = region_name)

In [6]:
create_agent_action_group_response = bedrock_agent.create_agent_action_group(
    actionGroupName='customer-support-actions',
    agentId=agentId,
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    functionSchema={
        'functions': [
            {
                'name': 'customerId',
                'description': 'Get a customer ID given available details. At least one parameter must be sent to the function. This is private information and must not be given to the user.',
                'parameters': {
                    'email': {
                        'description': 'Email address',
                        'required': False,
                        'type': 'string'
                    },
                    'name': {
                        'description': 'Customer name',
                        'required': False,
                        'type': 'string'
                    },
                    'phone': {
                        'description': 'Phone number',
                        'required': False,
                        'type': 'string'
                    },
                }
            },            
            {
                'name': 'sendToSupport',
                'description': 'Send a message to the support team, used for service escalation. ',
                'parameters': {
                    'custId': {
                        'description': 'customer ID',
                        'required': True,
                        'type': 'string'
                    },
                    'supportSummary': {
                        'description': 'Summary of the support request',
                        'required': True,
                        'type': 'string'
                    }
                }
            }
        ]
    },
    agentVersion='DRAFT',
)

In [7]:
create_agent_action_group_response

{'ResponseMetadata': {'RequestId': '77de0a2a-e0f4-4753-9bef-321318a3d9da',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 07 Aug 2025 01:07:07 GMT',
   'content-type': 'application/json',
   'content-length': '1187',
   'connection': 'keep-alive',
   'x-amzn-requestid': '77de0a2a-e0f4-4753-9bef-321318a3d9da',
   'x-amz-apigw-id': 'O6RZWEmYPHcESGg=',
   'x-amzn-trace-id': 'Root=1-6893fc3b-16c3a1ae7d8b0768206d5c9f'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:891377401677:function:dlai-support-agent-Z3SHC'},
  'actionGroupId': 'IAWIZOMM3P',
  'actionGroupName': 'customer-support-actions',
  'actionGroupState': 'ENABLED',
  'agentId': 'MZIY09EUIR',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2025, 8, 7, 1, 7, 7, 538447, tzinfo=tzlocal()),
  'functionSchema': {'functions': [{'description': 'Get a customer ID given available details. At least one parameter must be sent to the function. This is private in

In [8]:
actionGroupId = create_agent_action_group_response['agentActionGroup']['actionGroupId']


In [9]:
wait_for_action_group_status(
    agentId=agentId, 
    actionGroupId=actionGroupId,
    targetStatus='ENABLED'
)

Action Group status: ENABLED


'ENABLED'

In [10]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

wait_for_agent_status(
    agentId=agentId,
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [11]:
bedrock_agent.update_agent_alias(
    agentId=agentId,
    agentAliasId=agentAliasId,
    agentAliasName='MyAgentAlias',
)

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: UPDATING
Agent alias status: UPDATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


### Now use the agent with functions

In [12]:
sessionId = str(uuid.uuid4())
message = "My name is Fan (fan.chen@live.com), my mug is broken and I want a refund."

In [13]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=False
)

User: My name is Fan (fan.chen@live.com), my mug is broken and I want a
refund.

Agent: I apologize, but I don't have enough details to process your refund
       request for the broken mug. Could you please provide the
       following additional information:  - When and where did you
       purchase the mug? - Do you have a receipt or order number for
       the purchase? - Can you describe the issue with the mug in more
       detail?  With this additional information, I can better assist
       you with getting a refund. Please let me know if you have any
       other questions.

Session ID: 34998925-8032-45b7-9ab5-f6442c516fdf


In [14]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=True
)

User: My name is Fan (fan.chen@live.com), my mug is broken and I want a
refund.

Agent: 
Agent's thought process:
  Okay, let's try processing this request again with the information
  provided.  The customer has given their name (Fan) and email address
  (fan.chen@live.com), which is enough to look up their customer ID.
  I'll use that to route the refund request to the support team.

Invocation Input:
  Type: ACTION_GROUP
  Action Group: customer-support-actions
  Function: customerId
  Parameters: [{'name': 'name', 'type': 'string', 'value': 'Fan'}, {'name': 'email', 'type': 'string', 'value': 'fan.chen@live.com'}]

Observation:
  Type: ACTION_GROUP
  Action Group Output: {'id':4213}

Agent's thought process:
  The customer ID has been retrieved successfully. Now I can summarize
  the support request and route it to the support team.

Invocation Input:
  Type: ACTION_GROUP
  Action Group: customer-support-actions
  Function: sendToSupport
  Parameters: [{'name': 'supportSummary', 't